In [1]:
from models.helper import GetModel
from datasets.train_dataset import GSVCitiesDataset
from datasets.test_dataset import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0)
test_dataset = TestDataset()

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test')
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test')
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


# Phase 1

### Experiment 1: Avg Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [2]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_avg_model'

model = GetModel(aggregator='average')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2820959/111993283.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
/home/arda/anacon



+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 69.76 | 80.93  |
+----------+-------+--------+
R1: 0.6976104091079695
R5: 0.8093331665207056


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.70 | 50.80  |
+----------+-------+--------+
R1: 0.327
R5: 0.508


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 45.08 | 61.90  |
+----------+-------+--------+
R1: 0.4507936507936508
R5: 0.6190476190476191


### Experiment 2: Gem Pooling + Contrastive Loss + No Miner + SGD(lr=0.01, decay=1e-3)

In [3]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_gem_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3850795405.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.81 | 81.33  |
+----------+-------+--------+
R1: 0.7081196046540723
R5: 0.8133366695858877


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.20 | 48.40  |
+----------+-------+--------+
R1: 0.322
R5: 0.484


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.86 | 62.22  |
+----------+-------+--------+
R1: 0.42857142857142855
R5: 0.6222222222222222


# Phase 2

#### Experiment 2.1: Batch Hard Miner + Contrastive Loss

In [4]:
TRAIN = False

EXPERIMENT_NAME = 'contrastive_batchhard_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/2327040488.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 72.84 | 83.00  |
+----------+-------+--------+
R1: 0.7283873389215564
R5: 0.8299762292005505


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 32.50 | 48.60  |
+----------+-------+--------+
R1: 0.325
R5: 0.486


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.52 | 61.27  |
+----------+-------+--------+
R1: 0.49523809523809526
R5: 0.6126984126984127


#### Experiment 2.2: DistanceWeightedMiner + SupConLoss


In [5]:
TRAIN = False

EXPERIMENT_NAME = 'supcon_distanceweighted_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.DistanceWeightedMiner()
    loss_fn = losses.SupConLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/4286773646.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.81 | 82.30  |
+----------+-------+--------+
R1: 0.7081196046540723
R5: 0.8229700988364819


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 28.80 | 45.60  |
+----------+-------+--------+
R1: 0.288
R5: 0.456


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 43.49 | 57.46  |
+----------+-------+--------+
R1: 0.43492063492063493
R5: 0.5746031746031746


#### Experiment 2.3: PairMargin + FastAP

In [6]:
TRAIN = False

EXPERIMENT_NAME = 'fastap_pairmargin_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.PairMarginMiner()
    loss_fn = losses.FastAPLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/4279425740.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 71.34 | 82.46  |
+----------+-------+--------+
R1: 0.7133742024271237
R5: 0.8245965219567121


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 29.30 | 45.80  |
+----------+-------+--------+
R1: 0.293
R5: 0.458


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 40.63 | 53.97  |
+----------+-------+--------+
R1: 0.40634920634920635
R5: 0.5396825396825397


#### Experiment 2.4: No Miner + MultiSimilarityLoss

In [7]:
TRAIN = False

EXPERIMENT_NAME = 'multisimilarity_model'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = None
    loss_fn = losses.MultiSimilarityLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-3,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/1054108450.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 70.62 | 81.87  |
+----------+-------+--------+
R1: 0.7062429625922683
R5: 0.818716376829726


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 29.10 | 47.00  |
+----------+-------+--------+
R1: 0.291
R5: 0.47


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.86 | 61.59  |
+----------+-------+--------+
R1: 0.42857142857142855
R5: 0.6158730158730159


# Phase 3: Optimizer, LR, WD and LR Scheduler

## SGD, 1e-2, 1e-5

In [8]:
TRAIN = False

EXPERIMENT_NAME = 'SGD_1e2_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.01,  
                            weight_decay=1e-5,
                            momentum=0.9)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3927305494.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 73.98 | 84.22  |
+----------+-------+--------+
R1: 0.7397723007631678
R5: 0.8422369573376705


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 37.20 | 53.90  |
+----------+-------+--------+
R1: 0.372
R5: 0.539


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 50.48 | 66.98  |
+----------+-------+--------+
R1: 0.5047619047619047
R5: 0.6698412698412698


## 3.2 ADAMW, 1e-4, 1e-4

In [9]:
TRAIN = False

EXPERIMENT_NAME = 'AdamW_1e4_1e4'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.AdamW(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-4)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3593095194.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.03 | 85.25  |
+----------+-------+--------+
R1: 0.7502814963092707
R5: 0.8524959339421995


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 37.50 | 53.80  |
+----------+-------+--------+
R1: 0.375
R5: 0.538


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.52 | 64.76  |
+----------+-------+--------+
R1: 0.49523809523809526
R5: 0.6476190476190476


## 3.3 ADAM, 1e-4, 1e-5

In [10]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 30,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': custom_scheduler,
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/1025689770.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.14 | 85.74  |
+----------+-------+--------+
R1: 0.7514074815463531
R5: 0.8573752033028901


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 38.40 | 54.80  |
+----------+-------+--------+
R1: 0.384
R5: 0.548


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 50.16 | 65.71  |
+----------+-------+--------+
R1: 0.5015873015873016
R5: 0.6571428571428571


# LR SCHEDULER

## No Scheduler

In [11]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_no_scheduler'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        # 'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3495529622.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 75.83 | 85.59  |
+----------+-------+--------+
R1: 0.7582885024396346
R5: 0.8558738896534468


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 39.60 | 55.50  |
+----------+-------+--------+
R1: 0.396
R5: 0.555


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 52.06 | 65.71  |
+----------+-------+--------+
R1: 0.5206349206349207
R5: 0.6571428571428571


## Poly LR Scheduler

In [12]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_custom'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/1329618683.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 77.46 | 87.13  |
+----------+-------+--------+
R1: 0.7745527336419367
R5: 0.8712623545602403


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 42.60 | 58.40  |
+----------+-------+--------+
R1: 0.426
R5: 0.584


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 53.02 | 70.48  |
+----------+-------+--------+
R1: 0.5301587301587302
R5: 0.7047619047619048


## Cosine Annealing LR Scheduler

In [13]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_cosine'

model = GetModel(aggregator='gem')
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'cosine',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/54693177.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 74.99 | 85.39  |
+----------+-------+--------+
R1: 0.7499061678969098
R5: 0.8538721381208557


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 39.00 | 56.80  |
+----------+-------+--------+
R1: 0.39
R5: 0.568


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 46.67 | 63.49  |
+----------+-------+--------+
R1: 0.4666666666666667
R5: 0.6349206349206349


# MIXVPR

In [14]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=512)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3146773426.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 79.24 | 87.00  |
+----------+-------+--------+
R1: 0.7924433879644689
R5: 0.8700112598523708


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 53.60 | 65.30  |
+----------+-------+--------+
R1: 0.536
R5: 0.653


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 66.03 | 79.68  |
+----------+-------+--------+
R1: 0.6603174603174603
R5: 0.7968253968253968


In [ ]:
TRAIN = False

EXPERIMENT_NAME = 'Adam_1e4_1e5_mixvpr_1024_custom'

model = GetModel(aggregator='mixvpr', input_size=256, output_size=1024)
model = model.to('cuda')
if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'overfitting_detector': True,
        'patience': 45,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True
}
queries, database, predictions = eval_model(**tokyo_test_args)

# Phase 4: Depth

In [15]:
from models.helper import GetModel, GetMultiModel, GetDeAttNet
from datasets.train_dataset import GSVCitiesDataset
from datasets.test_dataset import TestDataset
from torch.utils.data import DataLoader
import torch
import os
from pytorch_metric_learning import losses, miners
from train import train_model
from evaluation import eval_model
from utils.lr_scheduler import custom_scheduler
import random
import numpy as np
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)


train_dataset = GSVCitiesDataset(generated_data_prob=0, multi_model=True)
test_dataset = TestDataset(multi_model=True)

num_workers = 16
train_dataloader = DataLoader(train_dataset, 
                              batch_size=100, 
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True
                              )

test_dataloader = DataLoader(test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
sf_xs_test_dataset = TestDataset(path='./data/sf_xs/test', multi_model=True)
sf_xs_test_dataloader = DataLoader(sf_xs_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )
tokyo_test_dataset = TestDataset(path='./data/tokyo_xs/test', multi_model=True)
tokyo_test_dataloader = DataLoader(tokyo_test_dataset, 
                             batch_size=100, 
                             shuffle=False, 
                             num_workers=num_workers, 
                             pin_memory=True
                             )


torch.cuda.empty_cache()

# Concat

In [16]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_512'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/405888062.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 79.49 | 88.49  |
+----------+-------+--------+
R1: 0.7949455773802077
R5: 0.8848992868760165


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 48.80 | 62.90  |
+----------+-------+--------+
R1: 0.488
R5: 0.629


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 65.08 | 82.86  |
+----------+-------+--------+
R1: 0.6507936507936508
R5: 0.8285714285714286


In [17]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_no_fusion_arda_1024'

model = GetMultiModel(aggregator='gem', use_fusion=False, output_size=1024)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/3060636273.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 80.11 | 88.87  |
+----------+-------+--------+
R1: 0.8010759414487677
R5: 0.8886525709996247


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 49.80 | 64.00  |
+----------+-------+--------+
R1: 0.498
R5: 0.64


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 63.17 | 81.90  |
+----------+-------+--------+
R1: 0.6317460317460317
R5: 0.819047619047619


# Depth+ mixvpr

In [18]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_512'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=512)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/2687052716.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 82.42 | 89.72  |
+----------+-------+--------+
R1: 0.8242211935443513
R5: 0.8971600150131365


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 57.20 | 70.50  |
+----------+-------+--------+
R1: 0.572
R5: 0.705


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 78.73 | 88.25  |
+----------+-------+--------+
R1: 0.7873015873015873
R5: 0.8825396825396825


In [20]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_depth_mixvpr_arda_1024'

model = GetMultiModel(aggregator='mixvpr', use_fusion=False, input_size=512, output_size=1024)
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/1770950872.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 82.82 | 90.25  |
+----------+-------+--------+
R1: 0.8282246966095334
R5: 0.9025397222569749


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 58.80 | 68.30  |
+----------+-------+--------+
R1: 0.588
R5: 0.683


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 76.19 | 88.89  |
+----------+-------+--------+
R1: 0.7619047619047619
R5: 0.8888888888888888


# DEATTNET

In [19]:
TRAIN = False

EXPERIMENT_NAME = 'multimodel_deattnet'

model = GetDeAttNet(aggregator='gem')
model = model.to('cuda')
# model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))

if TRAIN:   
    miner = miners.BatchHardMiner()
    loss_fn = losses.ContrastiveLoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                            lr=1e-4,  
                            weight_decay=1e-5)
    train_args = {
        'model': model,
        'loss_fn': loss_fn,
        'miner': miner,
        'optimizer': optimizer,
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'test_dataset': test_dataset,
        'num_epochs': 45,
        'patience': 45,
        'overfitting_detector': True,
        'verbose': True,
        'scheduler': 'custom',
        'job_name': EXPERIMENT_NAME,
        'multi_model': True
    }

if TRAIN:
    model, r1, r5 = train_model(**train_args)
else:
    model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))
    model.to('cuda')
    model.eval()

print('Evaluating on test sets')
print('-'*20)
sf_xs_val_args = {
    'model': model,
    'test_dataloader': test_dataloader,
    'test_dataset': test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_val_args)

sf_xs_test_args = {
    'model': model,
    'test_dataloader': sf_xs_test_dataloader,
    'test_dataset': sf_xs_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**sf_xs_test_args)


tokyo_test_args = {
    'model': model,
    'test_dataloader': tokyo_test_dataloader,
    'test_dataset': tokyo_test_dataset,
    'verbose': True,
    'multi_model': True
}
queries, database, predictions = eval_model(**tokyo_test_args)
torch.cuda.empty_cache()

Evaluating on test sets
--------------------


/tmp/ipykernel_2811704/1937420694.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./weights/{EXPERIMENT_NAME}.pth'))




+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 76.99 | 86.75  |
+----------+-------+--------+
R1: 0.76992368322282
R5: 0.8675090704366321


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 41.40 | 56.50  |
+----------+-------+--------+
R1: 0.414
R5: 0.565


+---------------------------+
| Performance on val_loader |
+----------+-------+--------+
|    K     |   1   |   5    |
+----------+-------+--------+
| Recall@K | 53.33 | 66.35  |
+----------+-------+--------+
R1: 0.5333333333333333
R5: 0.6634920634920635
